In [6]:
import pandas as pd


In [2]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/LeticiaSouza04/Bootcamp/main/UNdata_Export_20240412_202657352.csv"


In [5]:
dados_commodities = pd.read_csv(url_commodities)

print(dados_commodities)


dados = pd.read_csv('https://data.un.org/Data.aspx?d=ComTrade&f=_l1Code%3a1')

      Country or Area  Year        Commodity       Flow   Trade (USD)   
0         Afghanistan  2019  All Commodities     Export  8.704885e+08  \
1         Afghanistan  2019  All Commodities     Import  8.568014e+09   
2         Afghanistan  2019  All Commodities  Re-Export  6.655197e+06   
3         Afghanistan  2018  All Commodities     Import  7.406590e+09   
4         Afghanistan  2018  All Commodities  Re-Export  9.263097e+06   
...               ...   ...              ...        ...           ...   
11175        Zimbabwe  2000  All Commodities  Re-Export  3.202940e+07   
11176        Zimbabwe  2000  All Commodities     Export  1.607053e+09   
11177        Zimbabwe  2000  All Commodities     Import  2.016521e+09   
11178        Zimbabwe  1995  All Commodities     Import  2.658853e+09   
11179        Zimbabwe  1995  All Commodities     Export  1.845906e+09   

       Weight (kg) Quantity Name  Quantity  
0              NaN   No Quantity       NaN  
1              NaN   No Quantity 